In [22]:
import os
from crewai import Agent, Task, Crew, Process
from crewai_tools import CSVSearchTool
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from datetime import datetime

In [2]:
load_dotenv()

OPEN_AI_KEY = os.getenv("OPEN_AI_KEY")

csv_folder = r"CSVs"

In [3]:
etapas = """
<etapas>\n
Etapa 1 -
Etapa 2 -
Etapa 3 -
Etapa 4 -
Etapa 5 -
Etapa 6 -
Etapa 7 -
Etapa 8 -
Etapa 9 -
</etapas>\n
"""

In [4]:
solicitacoes = """
<solicitacoes>
1. OBJETIVOS - Identificação dos objetivos: Analise o histórico de conversas no whatsapp com base na <etapa> do projeto e resuma o que houve\n
2. GAP - Identificação do GAP:\n
3. METODOLOGIA - Extração Detahada da Metodologia de Trabalho: Identificação e Descrição da Metodologia:\n
4. DATASET - \n
5. RESULTADOS - \n
6. LIMITAÇÕES - \n 
7. CONCLUSÃO - \n
8. FUTURO - \n
9. AVALIAÇÃO -
</solicitacoes>
"""

In [5]:
controles = """
<controles>
NÍVEIS DE CONTROLE:
1. Entonação: Formal\n 
5. Foco: Você deve responder sempre com foco no objetivo da <etapa> buscando encontrar GAPs, atingir <metas> e cumprir <prazos>\n
6. Línga: Escreva sempre em português do Brasil, como brasileiros especialistas em gestão de negócios constumam escrever.\n
7. Controle de Sentimento: Neutro. Evite superlativos como: inovador, revolucionário, etc.\n
8. Nível de originalidade: 10, onde 1 é pouco original e 10 é muito original.\n 
9. Nível de abstração: 1, onde 1 é muito concreto e 10 é muito abstrato e irreal.\n
10. Tempo Verbal: Escreva no presente.
</controles>
"""

In [6]:
restricoes = """
<restricoes>
O QUE VOCÊ NÃO DEVE FAZER:
1. \n 
2. \n
3. \n
4. \n
5. \n 
6. 
</restricoes>
"""

In [7]:
# Modelo de relatório
template = """
<template>
Relatório do Grupo: <grupo> - Etapa: <etapa>\n
Dia: <dia>\n
Prazo: <prazo>
1. Objetivos: Objetivos da etapa\n 
2. Análise de sentimento: Análise de sentimentos dos <clientes>\n
3. GAPs: Os gaps identificados no atendimento e relação com cliente, nos objetivos e etapas do projeto\n
4. \n
5. Pendências: As pendências do dia e da etapa\n 
. Avaliações: Avaliações sobre o dia e etapa
</template>
"""

In [8]:
def save_to_txt(file_name, file):
    """
    Salva as saídas do agente em um arquivo .txt.

    Parameters:
    file_name (str): O nome do arquivo onde as saídas serão salvas.
    goal (str): O objetivo do agente.
    backstory (str): A história de fundo do agente.
    """
    with open(file_name, 'w', encoding='utf-8') as f:
        f.write(file)  # Grava apenas o objetivo do agente

In [9]:
agent_relator_goal = (
    "Ler CSVs e extrair informações específicas conforme definido nas solicitações em <solicitacoes>. "
    f"Gerar um relatório de acordo com o modelo especificado em <template>.\n {solicitacoes} {template}"
)

agent_relator_backstory = (
    "Você é um especialista nos processos de atendimento da Goods. "
    "Sua missão é analisar as conversas entre a equipe de trabalho e os clientes e extrair informações, "
    "identificar GAPs, entender o processo e as etapas do projeto, gerar insights e propor melhorias para alcançar, "
    "além dos objetivos principais, a satisfação do cliente. "
    "Sua função é fundamental para avaliar nosso trabalho e otimizar processos. "
    "Ao responder as solicitações delimitadas por <solicitacoes></solicitacoes>, você deve levar em consideração as "
    "definições de controle em <controle></controle> e as restrições em <restricoes></restricoes>.\n"
    f"{solicitacoes} {restricoes} {controles}"
)

# Exibindo os resultados
# save_to_txt('agent_relator_goal',agent_relator_goal)
save_to_txt('agent_relator_backstory',agent_relator_backstory)

In [10]:
def create_agent_relator(llm, tool):
    return Agent(
        role="CSV Reader",
        goal=agent_relator_goal,
        backstory=agent_relator_backstory,
        tools=[tool],
        verbose=True,
        memory=False,
        llm=llm
    )

In [11]:
agent_revisor_goal = (
    "Leia os dados extraídos pelo Agente Relator e verifique se o relatório foi produzido "
    "de acordo com o template proposto em <template>, com os dados solicitados em <solicitacoes>. "
    "Como resultado do seu trabalho, você deve retornar um relatório revisado no mesmo formato do "
    f"template proposto. {solicitacoes} {template}"
)

agent_revisor_backstory = (
    "Você é um especialista em revisão de relatórios. "
    "Sua função é garantir que os dados extraídos reflitam as solicitações definidas em <solicitacoes> "
    "e estejam formatados de acordo com o template proposto em <template>. "
    "Sua atenção aos detalhes assegura que os resultados finais sejam precisos "
    f"conforme as expectativas. {solicitacoes} {template}"
)

# Exibindo os resultados
# save_to_txt('agent_revisor_goal',agent_revisor_goal)
save_to_txt('agent_revisor_backstory',agent_revisor_backstory)

In [12]:
def create_agent_revisor(llm):
    return Agent(
        role="Revisor do relatório",
        goal=agent_revisor_goal,
        backstory=agent_revisor_backstory,
        verbose=True,
        memory=False,
        llm=llm
    )

In [13]:
relator_task_description = (
    "Leia o CSV e retorne um relatório com as solicitações definidas em <solicitacoes> "
    "usando o modelo definido em <template>. "
    f"{solicitacoes} {template}"
)

relator_task_expeted_output = (
    "Relatório com as solicitações definidas em <solicitacoes>"
    "usando o modelo definido em <template>. "
    f"{solicitacoes} {template}"
)

In [14]:
def relator_task(agent_relator):
    return Task(
        description=relator_task_description,
        expeted_output=relator_task_expeted_output,
        agent=agent_relator
    )

In [15]:
revisor_task_description = (
    "Revise o relatório produzido pelo agente revisor para garantir que ele esteja de acordo com o template definido em <template> " 
    " e contenha todas as informações solicitadas em <solicitacoes> "
    f"{solicitacoes} {template}"
)
revisor_task_expeted_output = (
    "Relatório revisado que esteja de acordo com o template definido em <template>"
    " e contenha todas as informações solicitações em <solicitacoes>"
    f"{solicitacoes} {template}"
)

In [16]:
def revisor_task(agent_revisor):
    return Task(
        description=relator_task_description,
        expeted_output=relator_task_expeted_output,
        agent=agent_revisor
    )

In [19]:
csv_files = [f for f in os.listdir(csv_folder) if f.endswith('.csv')]

In [20]:
csv_files

['test.csv']

In [21]:
# lista ára armazenar resultados
all_ralatorios = []

In [ ]:
for csv_file_name in csv_files:
    gpt = ChatOpenAI(model="gpt-4") # a mente do agente
    csv = os.path.join(csv_folder, csv_file_name)
    csv_tool = CSVSearchTool(csv)
    # relator
    agent_relator = create_agent_relator(gpt,csv_tool)
    task_relator = relator_task(agent_relator)
    # revisor
    agent_revisor = create_agent_revisor(gpt)
    task_revisor = revisor_task(agent_revisor)
    
    crew = Crew(
        agents=[agent_relator,agent_revisor],
        tasks=[task_relator,task_revisor],
        process=Process.sequential
    )
    
    ipt = {
        "solicitacoes": solicitacoes,
        "template": template,
        "restricoes": restricoes,
        "controles": controles,
    }
    
    results = crew.kickoff(inputs=ipt)
    
    #TODO armazenar resultados no DB
    
    all_ralatorios.append(results)

final_output = {"relatorios": all_ralatorios}

now = datetime.now
formatted_date = now.strftime("%d_%m_%Y")

file_name = f"relatorios_{formatted_date}.txt"

with open(file_name, 'w') as file:
    (final_output, file,)

print(f"Arquivo '{file_name}' foi criado com sucesso!")

# Saida 2
file_name2 = 'output.txt'

with open(file_name2, 'w') as file:
    for key, value in final_output.items():
        file.write(f'{key}: {value}\n')

print(f"Dados salvos em {file_name2}")